In [1]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 114 kB/s 


In [15]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import plotly.express as px

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
PROJECT_PATH = "/content/drive/MyDrive/projects/interesting_visuals/company_sentiment/"
companies = ["apple", "google", "microsoft", "amazon", "tesla", "meta", "netflix", "facebook", "alphabet"]

In [4]:
dfs = []
for company in companies:
    data = pd.read_json(PROJECT_PATH + f"data/{company}.json").query("sentiment != 0")

    company_res = pd.DataFrame({
            "positive": [sum(data["sentiment"] > 0)],
            "negative": [sum(data["sentiment"] < 0)],
            "company": company,
        }, index=[company]
    )
    dfs.append(company_res)

sentiment_counts = pd.concat(dfs)

sentiment_counts.loc["google", ["positive", "negative"]] = sentiment_counts.loc[["google", "alphabet"], ["positive", "negative"]].sum()
sentiment_counts.drop("alphabet", inplace=True)

sentiment_counts.loc["meta", ["positive", "negative"]] = sentiment_counts.loc[["meta", "facebook"], ["positive", "negative"]].sum()
sentiment_counts.drop("facebook", inplace=True)

sentiment_counts["frac_pos"] = sentiment_counts["positive"] / (sentiment_counts["positive"] + sentiment_counts["negative"])
sentiment_counts["frac_neg"] = sentiment_counts["negative"] / (sentiment_counts["positive"] + sentiment_counts["negative"])
sentiment_counts.sort_values("frac_pos", inplace=True, ascending=False)
sentiment_counts

,positive,negative,company,frac_pos,frac_neg
apple,12546,5342,apple,0.701364,0.298636
microsoft,10936,5481,microsoft,0.666139,0.333861
tesla,9531,7947,tesla,0.545314,0.454686
netflix,7689,7014,netflix,0.522954,0.477046
amazon,8997,8461,amazon,0.515351,0.484649
google,11545,11464,google,0.501760,0.498240
meta,13223,15198,meta,0.465255,0.534745


In [24]:
sentiment_pos = sentiment_counts[["company", "frac_pos"]].copy()
sentiment_pos.columns = ["company", "frac"]
sentiment_pos["Sentiment"] = "positive"

sentiment_neg = sentiment_counts[["company", "frac_neg"]].copy()
sentiment_neg.columns = ["company", "frac"]
sentiment_neg["Sentiment"] = "negative"

sentiment_df = pd.concat([sentiment_pos, sentiment_neg])
sentiment_df["company"] = sentiment_df["company"].str.title()
sentiment_df["frac"] = (sentiment_df["frac"] * 100).round(0)

sentiment_df

,company,frac,Sentiment
apple,Apple,70.0,positive
microsoft,Microsoft,67.0,positive
tesla,Tesla,55.0,positive
netflix,Netflix,52.0,positive
amazon,Amazon,52.0,positive
google,Google,50.0,positive
meta,Meta,47.0,positive
apple,Apple,30.0,negative
microsoft,Microsoft,33.0,negative
tesla,Tesla,45.0,negative


In [26]:
fig = px.bar(
    sentiment_df, 
    x="company", 
    y="frac", 
    color="Sentiment", 
    width=1400,
    color_discrete_map={
        'positive': '#2ca02c',
        'negative': '#d62728',
    },
    template="plotly_white"
)

fig.update_layout(
    title="",
    xaxis_title="",
    yaxis_title="Sentiment",
    font=dict(
        size=24,
    ),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
)

fig.show()

fig.write_image("out.png")